In [1]:
from __future__ import absolute_import, division,print_function, unicode_literals
import os
from matplotlib import pyplot as plt
%matplotlib inline
import tensorflow as tf

In [2]:
tf.__version__

'2.0.0'

# Dataset

In [3]:
train_dataset_url = "https://storage.googleapis.com/download.tensorflow.org/data/iris_training.csv"
train_dataset_fp = tf.keras.utils.get_file(fname=os.path.basename(train_dataset_url), origin=train_dataset_url)
print("Local copy of the dataset file: {}".format(train_dataset_fp))

Local copy of the dataset file: /home/ubuntu/.keras/datasets/iris_training.csv


In [4]:
!head -n5 {train_dataset_fp}

120,4,setosa,versicolor,virginica
6.4,2.8,5.6,2.2,2
5.0,2.3,3.3,1.0,1
4.9,2.5,4.5,1.7,2
4.9,3.1,1.5,0.1,0


In [5]:
# column order in CSV
column_names = ['sepal_length', 'sepal_width', 'petal_length', 'petal_width', 'species']
feature_names = column_names[:-1]
label_name = column_names[-1]
class_names = ['Iris setosa', 'Iris versicolor', 'Iris virginica']

In [6]:
batch_size = 32

train_dataset = tf.data.experimental.make_csv_dataset(
    train_dataset_fp,
    batch_size,
    column_names=column_names,
    label_name=label_name,
    num_epochs=1)
train_dataset

W1223 21:34:04.487841 140267636582144 deprecation.py:323] From /home/ubuntu/tf20/lib/python3.5/site-packages/tensorflow_core/python/data/experimental/ops/readers.py:521: parallel_interleave (from tensorflow.python.data.experimental.ops.interleave_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use `tf.data.Dataset.interleave(map_func, cycle_length, block_length, num_parallel_calls=tf.data.experimental.AUTOTUNE)` instead. If sloppy execution is desired, use `tf.data.Options.experimental_determinstic`.


<PrefetchDataset shapes: (OrderedDict([(sepal_length, (None,)), (sepal_width, (None,)), (petal_length, (None,)), (petal_width, (None,))]), (None,)), types: (OrderedDict([(sepal_length, tf.float32), (sepal_width, tf.float32), (petal_length, tf.float32), (petal_width, tf.float32)]), tf.int32)>

In [7]:
def pack_features_vector(features, labels):
    """Pack the features into a single array."""
    features = tf.stack(list(features.values()), axis=1)
    return features, labels

In [8]:
train_dataset = train_dataset.map(pack_features_vector)

In [9]:
features, labels = next(iter(train_dataset))
print(features[:5])


tf.Tensor(
[[7.7 3.8 6.7 2.2]
 [5.  3.3 1.4 0.2]
 [4.8 3.4 1.6 0.2]
 [5.7 2.8 4.5 1.3]
 [5.7 4.4 1.5 0.4]], shape=(5, 4), dtype=float32)


In [10]:
model = tf.keras.Sequential([
  tf.keras.layers.Dense(10, activation=tf.nn.relu, input_shape=(4,)),  # input shape required
  tf.keras.layers.Dense(10, activation=tf.nn.relu),
  tf.keras.layers.Dense(3)
])

In [11]:
# return a logits for each classe
predictions = model(features)
predictions[:5]

<tf.Tensor: id=193, shape=(5, 3), dtype=float32, numpy=
array([[ 1.1150118 ,  3.0859942 , -4.013404  ],
       [-0.17516336,  1.2679344 , -1.8137453 ],
       [-0.13174956,  1.2788097 , -1.8459784 ],
       [ 0.6149421 ,  2.118577  , -2.790837  ],
       [-0.22490457,  1.6073036 , -2.2978969 ]], dtype=float32)>

In [12]:
# converting these logits to proba:
tf.nn.softmax(predictions[:5])

<tf.Tensor: id=198, shape=(5, 3), dtype=float32, numpy=
array([[1.2219485e-01, 8.7708104e-01, 7.2411954e-04],
       [1.8422833e-01, 7.7998430e-01, 3.5787333e-02],
       [1.8945318e-01, 7.7642584e-01, 3.4121003e-02],
       [1.8079291e-01, 8.1320822e-01, 5.9988806e-03],
       [1.3562125e-01, 8.4731644e-01, 1.7062340e-02]], dtype=float32)>

In [16]:
print(tf.argmax(predictions, axis=1))
print(labels)

tf.Tensor([1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1], shape=(32,), dtype=int64)
tf.Tensor([2 0 0 1 0 2 2 1 1 2 0 1 1 1 2 0 0 1 1 0 2 2 0 1 2 2 1 2 1 0 2 0], shape=(32,), dtype=int32)


In [17]:
loss_object = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

def loss(model, x, y):
    y_ = model(x)

    return loss_object(y_true=y, y_pred=y_)


l = loss(model, features, labels)
print("Loss test: {}".format(l))


Loss test: 2.750363826751709


In [18]:
def grad(model, inputs, targets):
    with tf.GradientTape() as tape:
        loss_value = loss(model, inputs, targets)
    return loss_value, tape.gradient(loss_value, model.trainable_variables)

In [20]:
optimizer = tf.keras.optimizers.SGD(learning_rate=0.01)

In [21]:
loss_value, grads = grad(model, features, labels)

print("Step: {}, Initial Loss: {}".format(optimizer.iterations.numpy(),
                                          loss_value.numpy()))

optimizer.apply_gradients(zip(grads, model.trainable_variables))

print("Step: {},         Loss: {}".format(optimizer.iterations.numpy(),
                                          loss(model, features, labels).numpy()))


Step: 0, Initial Loss: 2.750363826751709
Step: 1,         Loss: 2.415905714035034
